In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/olidv10/olid-training-v1.0.tsv
/kaggle/input/olidv10/testset-levelb.tsv
/kaggle/input/olidv10/labels-levelb.csv
/kaggle/input/glove6b100dtxt/glove.6B.100d.txt


In [2]:
# Import
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split,KFold
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm import tqdm

from collections import Counter

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import spacy
import re
import os

import logging
from gensim.models import FastText
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras import utils
from keras.models import Model
from keras import Sequential
from keras.layers import Dropout, Conv1D, MaxPooling1D, LSTM,Concatenate, Dense, GlobalMaxPooling1D,GlobalAveragePooling1D, Lambda, Input, Bidirectional, GRU, concatenate, SpatialDropout1D, Reshape, merge
from keras.layers.embeddings import Embedding

import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from keras.preprocessing import text,sequence

#import GPyOpt

Using TensorFlow backend.


In [3]:
data = pd.read_csv('../input/olidv10/olid-training-v1.0.tsv', sep = '\t',engine='python',encoding = 'utf-8-sig')

data = data.fillna('NULL')
data.head(10)

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NULL
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NULL,NULL
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NULL
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NULL,NULL
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NULL
7,52415,@USER was literally just talking about this lo...,OFF,TIN,GRP
8,45157,@USER Buy more icecream!!!,NOT,NULL,NULL
9,13384,@USER Canada doesn’t need another CUCK! We alr...,OFF,TIN,IND


In [4]:
print('Dataset size:',len(data))
data.info()

Dataset size: 13240
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 5 columns):
id           13240 non-null int64
tweet        13240 non-null object
subtask_a    13240 non-null object
subtask_b    13240 non-null object
subtask_c    13240 non-null object
dtypes: int64(1), object(4)
memory usage: 517.3+ KB


In [5]:
# Text preprocessing:
lemmatizer = WordNetLemmatizer() 
stops = stopwords.words('english')
tokenizer = RegexpTokenizer(r'\w+')
def script_preprocessing(df):
    
    def removewords(text): # Remove these words.
        
        text = text.replace('@USER','',50)
        text = text.strip('URL')
        text = text.replace('&amp','',10)
        return text

    def get_wordnet_pos(tag):

        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    
    def pos_lemma(text):
        
        tokens = [i for i in tokenizer.tokenize(str(removewords(text)).lower()) if i not in stops] # if we don't convert text to str, it rises TypeError: expected string or bytes-like object   
        tagged = pos_tag(tokens)
        lemlist = [lemmatizer.lemmatize(i[0], get_wordnet_pos(i[1])) for i in tagged]
        lemmas = ' '.join(lemlist).lower()

        return lemmas
    
    def countuser(text):
        splitted_text = text.lower().split()
        user_count = 0
        for word in splitted_text:
            word = re.sub("[#@]","",word)
            word = re.sub("!"," !",word)
            word = re.sub("[?]"," ?",word)
            if(word == 'user'):
                user_count += 1
        return user_count
                        
    def finalize(df):
        
        df['pos_lemmatized'] = [pos_lemma(i) for i in df['tweet']]
        df['user_count'] = [countuser(i) for i in df['tweet']]
        return df
        
    return finalize(df)

In [6]:
result = script_preprocessing(data)
result.head()

,id,tweet,subtask_a,subtask_b,subtask_c,pos_lemmatized,user_count
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NULL,ask native american take,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,go home drunk maga trump2020,3
2,16820,Amazon is investigating Chinese employees who ...,NOT,NULL,NULL,amazon investigate chinese employee sell inter...,0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NULL,someone vetaken piece shit volcano,1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NULL,NULL,obama want liberal illegals move red state,2


In [7]:
max_ = result['user_count'].max()
result['user_count'] = result['user_count'].apply(lambda x:x/max_)
result.head()

,id,tweet,subtask_a,subtask_b,subtask_c,pos_lemmatized,user_count
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NULL,ask native american take,0.02
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,go home drunk maga trump2020,0.06
2,16820,Amazon is investigating Chinese employees who ...,NOT,NULL,NULL,amazon investigate chinese employee sell inter...,0.00
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NULL,someone vetaken piece shit volcano,0.02
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NULL,NULL,obama want liberal illegals move red state,0.04


In [8]:
# Selecting the valid data for subtask2
ind = []
for i in range(len(result['subtask_b'])):
    if result['subtask_b'][i] == 'NULL':
        ind.append(i)
result.drop(axis=0,labels = ind,inplace = True)
result.head()

,id,tweet,subtask_a,subtask_b,subtask_c,pos_lemmatized,user_count
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NULL,ask native american take,0.02
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,go home drunk maga trump2020,0.06
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NULL,someone vetaken piece shit volcano,0.02
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH,liberal kookoo,0.02
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NULL,oh no tough shit,0.04


In [9]:
# Data_b = result[result["subtask_b"].notna()]
X = result[['pos_lemmatized',"user_count"]]
y = result['subtask_b']
label_to_number = {'UNT':0,'TIN':1}
number_to_label = {v:k for k,v in label_to_number.items()}
Y = result.subtask_b.apply(lambda x:label_to_number[x])

In [10]:
# Combine oversampling and undersampling in order to have balanced dataset
def same_size_data(X_train, count_train, y_train, ratio_down_over_up=0.5):
    X_train = list(X_train)
    count_train = list(count_train)
    y_train = list(y_train)
  
    n_cat = len(Counter(y_train))
  
    sorted_counter = Counter(y_train).most_common()
    max_cat = sorted_counter[0][1]
    min_cat = sorted_counter[-1][1]

    target = min_cat + (1-ratio_down_over_up)*(max_cat - min_cat)

    for i in range(n_cat):
        diff = int(sorted_counter[i][1] - target)
        k = 0
        if diff > 0:
            rm = 0    
            while rm <= diff:
                if(y_train[k] == sorted_counter[i][0]):
                    X_train.pop(k)
                    y_train.pop(k)
                    count_train.pop(k)
                    rm += 1
                    k -=1
                k += 1
        else:
            ad = 0
            while ad <= -diff:
                if(y_train[k] == sorted_counter[i][0]):
                    X_train.append(X_train[k])
                    y_train.append(y_train[k])
                    count_train.append(count_train[k])
                    ad += 1
                k += 1

    return X_train, count_train, y_train

In [11]:
X_train_b, count_train_b, y_train_b = same_size_data(X.pos_lemmatized, X.user_count, Y, 0.2)
X_train_b, count_train_b, y_train_b = shuffle(X_train_b, count_train_b, y_train_b)

**Bag of words**

In [12]:
# Bag of words model.
all_words_list = []
for sent in data['pos_lemmatized']:
    for word in sent.split(' '):
        all_words_list.append(word)
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(5000)

word_features = [word for (word,count) in word_items]
print(word_features[:30])

#word_features = tokenizer.texts_to_sequences(word_features)

['liberal', 'gun', 'like', 'control', 'get', 'people', 'go', 'shit', 'antifa', 'say', 'fuck', 'maga', 'know', 'conservative', 'think', 'trump', 'one', 'make', 'u', 'want', 'right', 'need', 'good', 'woman', 'would', 'democrat', 'lie', 'see', 'time', 'take']


In [13]:
# TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
def TFIDFmatrix(X_train,word_features):
    vectorizer = TfidfVectorizer()
    fit = vectorizer.fit(word_features)
    X = vectorizer.transform(X_train)
    X = X.todense()
    return X
    
X_train = TFIDFmatrix(X_train_b,word_features)

In [14]:
def training(X,y,clf):
    kf = KFold(n_splits=5, shuffle=True, random_state = 52)
    iteration_index = 0
    acc_list_scores = []
    f1_list_scores = []
    for train_indexes, test_indexes in kf.split(X, y):
        iteration_index += 1

        X_train = X[train_indexes]
        y_train = y[train_indexes]

        X_test = X[test_indexes]
        y_test = y[test_indexes]

        #logreg = linear_model.LogisticRegression(C=1e5)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)

        current_acc = accuracy_score(y_test, y_predict)
        current_f1 = f1_score(y_test, y_predict, average = 'macro')
        print("Iteration #{0}: Accuracy : {1}, F-score : {2}".format(iteration_index, current_acc, current_f1))
        #print(classification_report(y_test, y_predict, target_names= ["UNT",'TIN']))
        acc_list_scores.append(current_acc)
        f1_list_scores.append(current_f1)
        

    print("Accuracy: {0}".format(np.mean(acc_list_scores)))
    print("F1-measure: {0}".format(np.mean(f1_list_scores)))
    return clf

In [15]:
## Logistic Regression
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
logreg = linear_model.LogisticRegression(C=1e5)
LRmodel = training(X_train,np.array(y_train_b),logreg)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Iteration #1: Accuracy : 0.8986749805144193, F-score : 0.897213252150763


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Iteration #2: Accuracy : 0.8931357254290172, F-score : 0.8922069544969415


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Iteration #3: Accuracy : 0.890015600624025, F-score : 0.8886688761014583


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Iteration #4: Accuracy : 0.8853354134165367, F-score : 0.8846475198059891


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Iteration #5: Accuracy : 0.9017160686427457, F-score : 0.900277069874948
Accuracy: 0.8937755577253487
F1-measure: 0.89260273448602


In [16]:
# NaiveBayes
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
NBmodel = training(X_train,np.array(y_train_b),clf)

Iteration #1: Accuracy : 0.8612626656274357, F-score : 0.8577718709986
Iteration #2: Accuracy : 0.8728549141965679, F-score : 0.8710661846595624
Iteration #3: Accuracy : 0.8806552262090483, F-score : 0.8789308166395595
Iteration #4: Accuracy : 0.8822152886115444, F-score : 0.8812930207173977
Iteration #5: Accuracy : 0.8806552262090483, F-score : 0.878227895723952
Accuracy: 0.8755286641707289
F1-measure: 0.8734579577478143


In [17]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=30)
RFmodel=training(X_train,np.array(y_train_b),clf)

Iteration #1: Accuracy : 0.936087295401403, F-score : 0.9356563412808305
Iteration #2: Accuracy : 0.9305772230889235, F-score : 0.9303258742240907
Iteration #3: Accuracy : 0.9266770670826833, F-score : 0.9262807336935244
Iteration #4: Accuracy : 0.9407176287051482, F-score : 0.940665497417909
Iteration #5: Accuracy : 0.9399375975039002, F-score : 0.9395319690854328
Accuracy: 0.9347993623564117
F1-measure: 0.9344920831403576


In [18]:
# GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
GBmodel = training(X_train,np.array(y_train_b),clf)

Iteration #1: Accuracy : 0.8160561184723305, F-score : 0.81240458771982
Iteration #2: Accuracy : 0.8198127925117005, F-score : 0.8172778445175394
Iteration #3: Accuracy : 0.8057722308892356, F-score : 0.8020843214609222
Iteration #4: Accuracy : 0.8081123244929798, F-score : 0.8059927360774818
Iteration #5: Accuracy : 0.7987519500780031, F-score : 0.7945191182281066
Accuracy: 0.8097010832888498
F1-measure: 0.8064557216007741


In [19]:
# Test function.
def testify(x_test,y_test,model):
    y_predict = model.predict(x_test)
    acc = accuracy_score(y_test,y_predict)
    #f1 = f1_score(y_test, y_predict, average = 'macro')
    print(classification_report(y_test, y_predict, target_names= ["UNT",'TIN']))
    print('acc:',acc)
    #print('f1:',f1)
    #return acc,f1


# Import Test_data.
testdata = pd.read_csv('../input/olidv10/testset-levelb.tsv', sep = '\t',engine='python',encoding = 'utf-8-sig')
testdata = testdata.fillna('NULL')
testdata = script_preprocessing(testdata)
x_test = TFIDFmatrix(testdata['pos_lemmatized'],word_features)
y_test = pd.read_csv('../input/olidv10/labels-levelb.csv',sep = ',',engine='python',header = None,encoding = 'utf-8-sig')
y_test = y_test[1].apply(lambda x:label_to_number[x])

In [20]:
# Model testing:
testify(x_test,y_test,LRmodel)
testify(x_test,y_test,NBmodel)
testify(x_test,y_test,GBmodel)
testify(x_test,y_test,RFmodel)

              precision    recall  f1-score   support

         UNT       0.20      0.37      0.26        27
         TIN       0.91      0.81      0.86       213

    accuracy                           0.76       240
   macro avg       0.56      0.59      0.56       240
weighted avg       0.83      0.76      0.79       240

acc: 0.7625
              precision    recall  f1-score   support

         UNT       0.22      0.48      0.30        27
         TIN       0.92      0.78      0.84       213

    accuracy                           0.75       240
   macro avg       0.57      0.63      0.57       240
weighted avg       0.84      0.75      0.78       240

acc: 0.7458333333333333
              precision    recall  f1-score   support

         UNT       0.26      0.81      0.40        27
         TIN       0.97      0.71      0.82       213

    accuracy                           0.72       240
   macro avg       0.61      0.76      0.61       240
weighted avg       0.89      0.72     

** Create word embedding.**

In [21]:
# Create word embedding.

maxlen = 10
# Tokenize each word in the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_b)
Xtr_seq = tokenizer.texts_to_sequences(X_train_b)
Xtr_seq = sequence.pad_sequences(Xtr_seq,maxlen=maxlen)
Xte_seq = tokenizer.texts_to_sequences(testdata['pos_lemmatized'])
Xte_seq = sequence.pad_sequences(Xte_seq,maxlen=maxlen)
preprocessed = tokenizer.sequences_to_texts(Xtr_seq)
sentences = [t.split() for t in preprocessed]
word_index = tokenizer.word_index

In [22]:
# Create the embedding matrix(using glove method)
EMBEDDING_FILE = '../input/glove6b100dtxt/glove.6B.100d.txt'
Embedding_size = 100
threshold = 0.4

def get_coefs(word,*arr):
    return word,np.asarray(arr,dtype='float32')

Embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))
max_features = len(word_index)+1
Embedding_matrix = np.zeros((max_features,Embedding_size))
for word,i in word_index.items():
    try:
        embedding_vector = Embeddings_index.get(word)
        if embedding_vector is not None:
            Embedding_matrix[i] = embedding_vector
    except KeyError:
        print(word)
        


In [23]:
# CNN
inp = Input(shape = (maxlen,))

x = Embedding(max_features, Embedding_size, weights = [Embedding_matrix], trainable = True)(inp)
x = SpatialDropout1D(0.5)(x)
x = Bidirectional(LSTM(128, return_sequences = True))(x)
x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])
x = Dense(10, activation = "relu")(x)

a = Dense(1, activation = "sigmoid")(x)

model_b = Model(inputs = inp, outputs = a)
model_b.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 100)      775200      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 10, 100)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 256)      234496      spatial_dropout1d_1[0][0]        
____________________________________________________________________________________________

In [24]:
model_b.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

X_tra, X_val, y_tra, y_val = train_test_split(Xtr_seq, y_train_b, train_size=0.8,random_state=233)
#model_b.fit(np.array(X_train_b), y_train_b, validation_data=(np.array(testdata['pos_lemmatized']),y_test), epochs = 3)
model_b.fit(np.array(X_tra), y_tra, validation_data=(X_val,y_val), epochs = 4)

y_pred_b = model_b.predict(Xte_seq)
y_pred_b = y_pred_b > threshold
print(classification_report(y_test, y_pred_b, target_names= ["UNT",'TIN']))


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5128 samples, validate on 1283 samples
Epoch 1/4
5128/5128 [==============================] - 9s 2ms/step - loss: 0.6554 - accuracy: 0.6092 - val_loss: 0.6489 - val_accuracy: 0.6220
Epoch 2/4
5128/5128 [==============================] - 5s 990us/step - loss: 0.6016 - accuracy: 0.6722 - val_loss: 0.5660 - val_accuracy: 0.6867
Epoch 3/4
5128/5128 [==============================] - 5s 991us/step - loss: 0.5185 - accuracy: 0.7440 - val_loss: 0.4515 - val_accuracy: 0.7857
Epoch 4/4
5128/5128 [==============================] - 5s 1ms/step - loss: 0.4194 - accuracy: 0.8153 - val_loss: 0.3520 - val_accuracy: 0.8223
              precision    recall  f1-score   support

         UNT       0.28      0.41      0.33        27
         TIN       0.92      0.86      0.89       213

    accuracy                           0.81       240
   macro avg       0.60      0.64      0.61       240
weighted avg       0.85      0.81      0.83       240

